## Inheritance

In [107]:
class Card():
    """Playing card"""
    
    rankMap = [("%d" % i) for i in range(13)] # Class attributes
    (rankMap[0],rankMap[1],*rankMap[11:13]) = ('None','A','J','Q','K')
    # Interestingly, it seems that I can only use * in the left side once, so I cannot * both vectors
    # print(rankMap)
    suitMap = ['♣','♢','♡','♠']
    # ['None', 'A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']
    decoderRanks = {'A':1, 'J':11, 'Q':12, 'K':13}
    decoderSuit = {'c':0, 'd':1, 'h':2, 's':3}    
    
    def __init__(self,suit=0,rank=1):
        if isinstance(suit, int):
            self.suit = suit # Instance attributes
        elif type(suit)==str:
            self.suit = Card.decoderSuit[suit]
        else:
            raise ValueErorr('Illegal card suit declaration')
        
        if isinstance(rank, int):
            self.rank = rank
        elif type(rank)==str:
            if rank in Card.decoderRanks.keys():
                self.rank = Card.decoderRanks[rank]
            else:
                self.rank = int(rank) # Evaluate string
        else:
            raise ValueError('Illegal card rank declaration')
        
    def __str__(self):
        return "%s%s" % (Card.rankMap[self.rank] , Card.suitMap[self.suit])
           
    def __eq__(self,other):
        return (self.rank==other.rank) & (self.suit==other.suit)
    
    def __gt__(self,other): 
        # They removed __cmp__ in Python 3, replacing it with: eq, ne, lt, le, gt, ge
        # However, Python somehow guesses how > will behave knowing <. So the minimum includes EQ, GT & GE
        # And then there's also __repr__ that is used for eval(). In this case, will be "Card(0,1)" - but with actual values
        return (self.suit, self.rank) > (other.suit, other.rank) # Uses the fancy lexicographicish way tuples are compared
        
    def __ge__(self,other):
        return self.__gt__(other) | self.__eq__(other)
        

a = Card(0,2)
print(a)
print(Card('c','Q'))
print(a==Card(0,3))
print(a!=Card(0,3))
print(a>Card(0,3))
print(a<Card(0,3))
print(a<=Card(0,3))
print(a>=Card(0,3))

2♣
Q♣
False
True
False
True
True
False


In [2]:
import random

In [88]:
class Deck(object):
    """Deck of cards"""
    
    def __init__(self):
        self.cards = []
        self.label = 'Deck'
        for s in range(4):
            for r in range(1,14):
                self.cards.append(Card(s,r))
                
    def __str__(self):
        res = []
        res.append('%s[%d]:' % (self.label,len(self.cards)))
        for card in self.cards:
            # res = res+str(card)+'|' # This is slow, apparently
            res.append(str(card)) # Fast: first a list, then join all with a delim
        return ' '.join(res)
    
    def pop(self): # Returns last card and updates the array (actual pop)
        return self.cards.pop()
    
    def add(self,card):
        self.cards.append(card)
        
    def shuffle(self):
        random.shuffle(self.cards)
        
    def deal(self,hand,num=1):
        for i in range(num):
            hand.add(self.pop())
            
    def len(self):
        return len(self.cards)
    
                
d = Deck()
print(d)
print(d.pop())
d.add(Card(2,2))
print(d)
d.shuffle()
print(d)

Deck[52]: A♣ 2♣ 3♣ 4♣ 5♣ 6♣ 7♣ 8♣ 9♣ 10♣ J♣ Q♣ K♣ A♢ 2♢ 3♢ 4♢ 5♢ 6♢ 7♢ 8♢ 9♢ 10♢ J♢ Q♢ K♢ A♡ 2♡ 3♡ 4♡ 5♡ 6♡ 7♡ 8♡ 9♡ 10♡ J♡ Q♡ K♡ A♠ 2♠ 3♠ 4♠ 5♠ 6♠ 7♠ 8♠ 9♠ 10♠ J♠ Q♠ K♠
K♠
Deck[52]: A♣ 2♣ 3♣ 4♣ 5♣ 6♣ 7♣ 8♣ 9♣ 10♣ J♣ Q♣ K♣ A♢ 2♢ 3♢ 4♢ 5♢ 6♢ 7♢ 8♢ 9♢ 10♢ J♢ Q♢ K♢ A♡ 2♡ 3♡ 4♡ 5♡ 6♡ 7♡ 8♡ 9♡ 10♡ J♡ Q♡ K♡ A♠ 2♠ 3♠ 4♠ 5♠ 6♠ 7♠ 8♠ 9♠ 10♠ J♠ Q♠ 2♡
Deck[52]: 10♡ 5♢ A♡ 5♣ 5♠ 4♡ 10♣ K♢ 6♣ J♣ 10♠ 10♢ 3♢ 9♠ 3♠ A♢ 7♡ J♢ 8♠ 7♠ J♠ 8♡ K♡ Q♢ 8♣ 2♣ 2♡ 7♣ Q♠ 3♡ 5♡ Q♡ 8♢ 9♢ J♡ 7♢ 2♢ 2♡ Q♣ 4♢ 4♠ A♣ 6♡ 9♡ 4♣ 6♢ 3♣ 2♠ A♠ 6♠ K♣ 9♣


In [5]:
def find_defining_class(obj,_method): # Author's advice on localizing which class a method is inhereted from
    for _type in type(obj).mro(): # mro() stands for Method Resolution Order
        if _method in _type.__dict__:
            return _type

find_defining_class(d, 'shuffle')

__main__.Deck

In [313]:
class Hand(Deck): # Inheritance: inherits to the main Deck class
    """Hand of cards"""
    
    def __init__(self,content=''):
        self.cards = []
        self.label = ''
        if len(content)>0:
            clist = content.split(' ')
            for code in clist:
                self.add(Card(code[-1],code[:-1])) # First suit, then rank
        self.classify()
        
    def take(self,deck,n=1):
        deck.deal(self,n)
        
    def count_chain(ranks):
        """Wrapper for count_chain_inner that checks Ace as both 1 and 14"""
        ranks2 = [i if i!=1 else 14 for i in ranks]
        return max(Hand.count_chain_inner(ranks) , Hand.count_chain_inner(ranks2))
        
    def count_chain_inner(ranks):
        """Looks for straights in a list of ranks"""
        ranks = list(set(ranks))
        ranks.sort()
        longest_chain = 0
        current_chain = 0
        for _ in range(len(ranks)-1):
            if ranks[_+1]-ranks[_]==1: # Count correct transitions from one card to next card
                current_chain += 1
                if longest_chain<current_chain:
                    longest_chain = current_chain
            else:
                current_chain = 0
        return longest_chain+1 # We add 1 because 1 transition is 2 cards etc.
        
    def classify(self):
        """Classifies a poker hand"""
        if len(self.cards)==0:
            return
        self.cards.sort() # Because we defined all <>, we can sort cards! May be helpful.
        histR = {}
        histS = {}
        for c in self.cards:
            histR[c.rank] = histR.get(c.rank,0)+1
            histS[c.suit] = histS.get(c.suit,0)+1
        v = [histR[_] for _ in histR.keys()]
        
        longest_chain = []
        for i_suit in range(4):
            longest_chain.append(0) 
            longest_chain[i_suit] = Hand.count_chain([c.rank for c in self.cards if c.suit==i_suit])
        single_color_chain = max(longest_chain)        
        overall_chain = Hand.count_chain([c.rank for c in self.cards])

        if single_color_chain>=5:
            self.label = 'Straight Flush'
        elif max(v)==4:
            self.label = 'Four'
        elif sum([_==3 for _ in v])==1 & sum([_==2 for _ in v])==1:
            self.label = 'House'
        elif max([histS[_] for _ in histS.keys()])==5:
            self.label = 'Flush'
        elif overall_chain>=5:
            self.label = 'Straight'
        elif max(v)==3:
            self.label = 'Three'
        elif sum([_==2 for _ in v])==2:
            self.label = '2 pairs'
        elif sum([_==2 for _ in v])==1:
            self.label = 'Pair'
        else:
            self.label = 'none'

In [314]:
print(Hand('9c 6d 8d Qd 5h 7h 8s'))

Straight[7]: 9♣ 6♢ 8♢ Q♢ 5♡ 7♡ 8♠


In [315]:
handSize = 7
type_hist = {'Pair':0, '2 pairs':0, 'Three':0, 'Straight':0, 'Flush':0, 'House':0, 'Four':0, 'Straight Flush':0}
counter = 0
for j in range(10000):
    d = Deck()
    d.shuffle()
    hs_in_d = int(d.len()/handSize)
    for i in range(hs_in_d):
        counter += 1
        h = Hand()
        h.take(d,handSize)
        h.classify()
        type_hist[h.label] = type_hist.get(h.label,0)+1
        if counter<10:
            print(h)
        
print('\n'.join(["%15s: \t %8.5f" % (i,100.0*type_hist[i]/counter) for i in type_hist.keys()]))

Pair[7]: 3♣ 5♣ 5♢ 9♡ 4♠ 6♠ 8♠
Pair[7]: 8♣ 2♢ 9♢ Q♢ 3♡ J♡ 2♠
Pair[7]: 6♣ 8♢ 4♡ 7♡ 7♠ 10♠ J♠
Pair[7]: 2♣ 7♣ 10♣ Q♣ 3♢ 10♢ A♡
2 pairs[7]: 4♣ 4♢ 10♡ K♡ 3♠ 5♠ K♠
2 pairs[7]: 9♣ J♣ 2♡ 6♡ Q♡ 9♠ Q♠
Three[7]: A♣ A♢ 7♢ J♢ K♢ 5♡ A♠
2 pairs[7]: J♢ 3♡ J♡ 3♠ 4♠ 6♠ 9♠
Pair[7]: 7♣ Q♣ 2♢ 8♢ A♡ 9♡ A♠
           Pair: 	 43.88429
        2 pairs: 	 21.47571
          Three: 	  4.98429
       Straight: 	  4.62000
          Flush: 	  2.92000
          House: 	  2.48857
           Four: 	  0.16571
 Straight Flush: 	  0.02571
           none: 	 19.43571


In [302]:
shift = random.randint(1,13-5)
ranks = [_+shift for _ in range(5)] + [random.randint(1,13) for _ in range(2)]
suits = [random.randint(0,3) for _ in range(7)]
h = Hand()
h.cards = [Card(suits[_],ranks[_]) for _ in range(7)]
h.shuffle()
h.classify()
print(h)

Pair[7]: 9♣ 6♢ 8♢ Q♢ 5♡ 7♡ 8♠
